<a href="https://colab.research.google.com/github/jiyeon712/Rainfall_model/blob/main/Rainfall_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 전처리

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/drive/MyDrive/weather_contest/rainfall_train.csv',index_col=0)
test = pd.read_csv('/content/drive/MyDrive/weather_contest/rainfall_test.csv')

train.columns = train.columns.str.replace('rainfall_train.', '')
test.columns = test.columns.str.replace('rainfall_test.', '')
train.shape, test.shape

((1457252, 21), (122000, 20))

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1457252 entries, 1 to 1457252
Data columns (total 21 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fc_year         1457252 non-null  object 
 1   fc_month        1457252 non-null  int64  
 2   fc_day          1457252 non-null  int64  
 3   fc_hour         1457252 non-null  int64  
 4   stn4contest     1457252 non-null  object 
 5   dh              1457252 non-null  int64  
 6   ef_year         1457252 non-null  object 
 7   ef_month        1457252 non-null  int64  
 8   ef_day          1457252 non-null  int64  
 9   ef_hour         1457252 non-null  int64  
 10  v01             1457252 non-null  int64  
 11  v02             1457252 non-null  int64  
 12  v03             1457252 non-null  int64  
 13  v04             1457252 non-null  int64  
 14  v05             1457252 non-null  int64  
 15  v06             1457252 non-null  int64  
 16  v07             1457252 non-null  int64  

In [ ]:
train.head(3)

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,...,v02,v03,v04,v05,v06,v07,v08,v09,vv,class_interval
1,A,5,1,9,STN001,3,A,5,1,12,...,0,0,0,0,0,0,0,0,0.0,0
2,A,5,1,9,STN001,6,A,5,1,15,...,0,0,0,0,0,0,0,0,0.0,0
3,A,5,1,9,STN001,9,A,5,1,18,...,0,0,0,0,0,0,0,0,0.0,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1457252 entries, 1 to 1457252
Data columns (total 21 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   fc_year         1457252 non-null  object 
 1   fc_month        1457252 non-null  int64  
 2   fc_day          1457252 non-null  int64  
 3   fc_hour         1457252 non-null  int64  
 4   stn4contest     1457252 non-null  object 
 5   dh              1457252 non-null  int64  
 6   ef_year         1457252 non-null  object 
 7   ef_month        1457252 non-null  int64  
 8   ef_day          1457252 non-null  int64  
 9   ef_hour         1457252 non-null  int64  
 10  v01             1457252 non-null  int64  
 11  v02             1457252 non-null  int64  
 12  v03             1457252 non-null  int64  
 13  v04             1457252 non-null  int64  
 14  v05             1457252 non-null  int64  
 15  v06             1457252 non-null  int64  
 16  v07             1457252 non-null  int64  

In [ ]:
train = train[train['class_interval'] != -999].copy() # 8490행 제거
train = train.drop('vv',axis=1)
train.shape

(1448762, 20)

In [ ]:
train.head()

,fc_year,fc_month,fc_day,fc_hour,stn4contest,dh,ef_year,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,class_interval
1,A,5,1,9,STN001,3,A,5,1,12,0,0,0,0,0,0,0,0,0,0
2,A,5,1,9,STN001,6,A,5,1,15,0,0,0,0,0,0,0,0,0,0
3,A,5,1,9,STN001,9,A,5,1,18,0,0,0,0,0,0,0,0,0,0
4,A,5,1,9,STN001,12,A,5,1,21,0,0,0,0,0,0,0,0,0,0
5,A,5,1,9,STN001,15,A,5,2,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test.shape

(122000, 20)

In [ ]:
# test데이터 -999값 처리

test_out = test[test['class_interval'] == -999]['class_interval']
test = test[test['class_interval'] != -999]
test = test.drop(['class_interval'], axis=1)

In [ ]:
test.shape

(121870, 19)

In [ ]:
# 범주형 따로 분리
o_train = train.select_dtypes(include = 'object').copy()
f_train = train.select_dtypes(exclude = 'object').copy()
o_test = test.select_dtypes(include = 'object').copy()
f_test = test.select_dtypes(exclude = 'object').copy()

In [ ]:
o_train.head()

,fc_year,stn4contest,ef_year
1,A,STN001,A
2,A,STN001,A
3,A,STN001,A
4,A,STN001,A
5,A,STN001,A


In [ ]:
# 범주형 데이터 원핫인코딩
cols = ['fc_year','stn4contest','ef_year']
o_train = pd.get_dummies(o_train).astype(int)
o_test = pd.get_dummies(o_test).astype(int)
o_test = o_test.reindex(columns=o_train.columns, fill_value=0)

### 정규화

In [ ]:
f_train.head()

,fc_month,fc_day,fc_hour,dh,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,class_interval
1,5,1,9,3,5,1,12,0,0,0,0,0,0,0,0,0,0
2,5,1,9,6,5,1,15,0,0,0,0,0,0,0,0,0,0
3,5,1,9,9,5,1,18,0,0,0,0,0,0,0,0,0,0
4,5,1,9,12,5,1,21,0,0,0,0,0,0,0,0,0,0
5,5,1,9,15,5,2,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.preprocessing import StandardScaler

cols = ['fc_month', 'fc_day', 'fc_hour', 'dh', 'ef_month', 'ef_day', 'ef_hour',
       'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09']

scaler = StandardScaler()
f_train[cols] = scaler.fit_transform(f_train[cols])
f_test[cols] = scaler.transform(f_test[cols])

f_train.head()

,fc_month,fc_day,fc_hour,dh,ef_month,ef_day,ef_hour,v01,v02,v03,v04,v05,v06,v07,v08,v09,class_interval
1,-1.428899,-1.671065,-0.999724,-1.709704,-1.499252,-1.668280,0.216247,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
2,-1.428899,-1.671065,-0.999724,-1.666389,-1.499252,-1.668280,0.652615,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
3,-1.428899,-1.671065,-0.999724,-1.623073,-1.499252,-1.668280,1.088984,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
4,-1.428899,-1.671065,-0.999724,-1.579758,-1.499252,-1.668280,1.525352,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
5,-1.428899,-1.671065,-0.999724,-1.536442,-1.499252,-1.555365,-1.529227,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0


In [ ]:
# 범주형, 수치형 데이터 다시 합치기

train = pd.concat([o_train, f_train],axis = 1)
test = pd.concat([o_test,f_test],axis = 1)

In [ ]:
train.head()

,fc_year_A,fc_year_B,fc_year_C,stn4contest_STN001,stn4contest_STN002,stn4contest_STN003,stn4contest_STN004,stn4contest_STN005,stn4contest_STN006,stn4contest_STN007,...,v01,v02,v03,v04,v05,v06,v07,v08,v09,class_interval
1,1,0,0,1,0,0,0,0,0,0,...,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
2,1,0,0,1,0,0,0,0,0,0,...,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
3,1,0,0,1,0,0,0,0,0,0,...,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
4,1,0,0,1,0,0,0,0,0,0,...,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0
5,1,0,0,1,0,0,0,0,0,0,...,-1.03704,-0.940039,-0.801697,-0.691678,-0.588054,-0.446512,-0.326327,-0.191514,-0.12541,0


In [ ]:
x_train = train.iloc[:,:-1]
target = train['class_interval']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, target, test_size = 0.2, random_state = 42)

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(1159009, 42) (289753, 42)
(1159009,) (289753,)


In [ ]:
# 가중치 부여

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

### RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import numpy as np

x_train = x_train.values
x_test = x_test.values

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
x_train.shape

(1159009, 42, 1)

In [ ]:
y_train.shape

(1159009,)

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=20, activation='relu',
                    input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, class_weight=class_weights)

Epoch 1/10
28976/28976 [==============================] - 672s 23ms/step - loss: 2.3025 - accuracy: 0.2049 - val_loss: 2.3091 - val_accuracy: 0.2571
Epoch 2/10
28976/28976 [==============================] - 676s 23ms/step - loss: 2.3004 - accuracy: 0.1650 - val_loss: 2.2883 - val_accuracy: 0.0977
Epoch 3/10
 9885/28976 [=========>....................] - ETA: 6:53 - loss: 2.2930 - accuracy: 0.1882

KeyboardInterrupt: 

In [ ]:
model.save('/content/drive/MyDrive/weather_contest/model/rnn_model3.h5')

In [ ]:
# 예측
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 결과 출력
from sklearn.metrics import classification_report
print(classification_report(y_true_classes, y_pred_classes))

9055/9055 [==============================] - 62s 7ms/step


ValueError: `axis` must be fewer than the number of dimensions (1)

In [ ]:
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
y_pred = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred, axis=1)

3809/3809 [==============================] - 20s 5ms/step


In [ ]:
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['class_interval']
y_pred.value_counts()

class_interval
0                 121139
5                    683
7                     28
8                     20
Name: count, dtype: int64

In [ ]:
X_test_out = pd.DataFrame(X_test_out)
X_test_out.columns = ['class_interval']
X_test_out.head(2)

,class_interval
44,-999.0
120,-999.0


In [ ]:
submit = pd.concat([y_pred, X_test_out]).sort_index()
print(submit)

        class_interval
0                  0.0
1                  0.0
2                  0.0
3                  0.0
4                  0.0
...                ...
121865             0.0
121866             0.0
121867             0.0
121868             0.0
121869             0.0

[122000 rows x 1 columns]


In [ ]:
X_test = pd.read_csv('/content/drive/MyDrive/weather_contest/rainfall_test.csv')
X_test.drop(columns=['rainfall_test.class_interval'], inplace=True)
submit = submit.reset_index(drop=True)
X_test['rainfall_test.class_interval'] = submit['class_interval'].values
X_test['rainfall_test.class_interval'] = X_test['rainfall_test.class_interval'].astype(int)
X_test['rainfall_test.class_interval'].value_counts()

rainfall_test.class_interval
 0      121139
 5         683
-999       130
 7          28
 8          20
Name: count, dtype: int64

In [ ]:
X_test.to_csv('240120.csv')

In [ ]:
model.save('/content/drive/MyDrive/weather_contest/model/rnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
